In [1]:
%matplotlib inline
%run ../../import_envs.py
print('probtorch:', probtorch.__version__, 
      'torch:', torch.__version__, 
      'cuda:', torch.cuda.is_available())

probtorch: 0.0+5a2c637 torch: 1.0.0 cuda: True


In [2]:
data_path = "../gmm_dataset_c5k"
Data = torch.from_numpy(np.load(data_path + '/obs.npy')).float()
NUM_SEQS, N, D = Data.shape
K = 3 ## number of clusters

## Model Parameters
SAMPLE_SIZE = 10
NUM_HIDDEN_GLOBAL = 16
NUM_HIDDEN_LOCAL = 16
NUM_LAYERS = 1

## Training Parameters
BATCH_SIZE = 20
NUM_BATCHES = int((NUM_SEQS / BATCH_SIZE))
NUM_EPOCHS = 500
LEARNING_RATE = 5 * 1e-4

CUDA = torch.cuda.is_available()
PATH = 'oneshot-lstm'
DEVICE = torch.device('cuda:0')

## wrap up the parameters
Train_Params = (NUM_EPOCHS, NUM_BATCHES, NUM_SEQS, CUDA, DEVICE, PATH)
Model_Params = (N, K, D, SAMPLE_SIZE, BATCH_SIZE)

In [3]:
from lstm_model import *
from local_gibbs import *
oneshot_eta = LSTM_eta(K, D, BATCH_SIZE, SAMPLE_SIZE, NUM_HIDDEN_GLOBAL, NUM_HIDDEN_LOCAL, CUDA, DEVICE)
enc_z = Gibbs_z(K, CUDA, DEVICE)
if CUDA:
    oneshot_eta.cuda().to(DEVICE)
optimizer =  torch.optim.Adam(list(oneshot_eta.parameters()),lr=LEARNING_RATE, betas=(0.9, 0.99))

models = (oneshot_eta, enc_z)

In [4]:
from eubo_baseline import *
train_baseline(Eubo_baseline, optimizer, Data, Train_Params, Model_Params, models)

RuntimeError: Trying to backward through the graph a second time, but the buffers have already been freed. Specify retain_graph=True when calling backward the first time.